In [ ]:
import matplotlib.pyplot as plt
import json
import os

# Fungsi untuk menyimpan data
def save_data(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

# Fungsi untuk membaca data
def load_data(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            return json.load(file)
    else:
        return None

# Fungsi login
def login():
    username = input("Masukkan username: ")
    password = input("Masukkan password: ")

    if username in users and users[username]['password'] == password:
        return users[username]
    else:
        print("Login gagal. Cek kembali username dan password.")
        return None

# Fungsi register
def register():
    username = input("Masukkan username: ")

    if username not in users:
        password = input("Masukkan password: ")
        role = input("Pilih peran (admin/user): ")

        users[username] = {'password': password, 'role': role, 'cart': []}
        save_data(users, 'users.json')

        print("Registrasi berhasil.")
        return users[username]
    else:
        print("Username sudah terdaftar.")
        return None

# Fungsi untuk menampilkan menu admin
def admin_menu(user):
    while True:
        print("\n===== Menu Admin =====")
        print("1. Tampilkan list barang")
        print("2. Tambah barang")
        print("3. Perbarui stok dan harga barang")
        print("4. Hapus barang")
        print("5. Cari barang")
        print("6. Visualisasi hasil penjualan")
        print("7. Laporan penjualan")
        print("8. Keluar")

        choice = input("Pilih menu (1-8): ")

        if choice == '1':
            show_products()
        elif choice == '2':
            add_product()
        elif choice == '3':
            update_product()
        elif choice == '4':
            delete_product()
        elif choice == '5':
            search_product()
        elif choice == '6':
            visualize_sales()
        elif choice == '7':
            sales_report()
        elif choice == '8':
            break
        else:
            print("Pilihan tidak valid. Coba lagi.")

# Fungsi untuk menampilkan menu user
def user_menu(user):
    while True:
        print("\n===== Menu User =====")
        print("1. Masukkan pesanan")
        print("2. Total Pesanan Anda")
        print("3. Diskon Potongan Harga")
        print("4. Keluar")

        choice = input("Pilih menu (1/2/3/4): ")

        if choice == '1':
            place_order(user)
        elif choice == '2':
            calculate_total(user)
        elif choice == '3' :
            apply_discount(user)
        elif choice == '4':
            # user['cart'] = []
            # user.pop('total', None)
            break
        else:
            print("Pilihan tidak valid. Coba lagi.")

# Fungsi untuk menampilkan daftar barang
def show_products():
    for i, product in enumerate(products):
        print(f"{i + 1}. {product['name']} - Stok: {product['stock']}, Harga: {product['price']}")

# Fungsi untuk menambah barang
def add_product():
    name = input("Masukkan nama barang: ")
    stock = int(input("Masukkan stok barang: "))
    price = float(input("Masukkan harga barang: "))

    product = {'name': name, 'stock': stock, 'price': price}
    products.append(product)
    save_data(products, 'products.json')

    print("Barang berhasil ditambahkan.")

# Fungsi untuk memperbarui stok dan harga barang
def update_product():
    show_products()
    product_index = int(input("Masukkan indeks barang yang ingin diperbarui: "))

    if 0 <= product_index < len(products):
        new_stock = int(input(f"Stok baru ({products[product_index]['stock']}): ") or products[product_index]['stock'])
        new_price = float(input(f"Harga baru ({products[product_index]['price']}): ") or products[product_index]['price'])

        products[product_index]['stock'] = new_stock
        products[product_index]['price'] = new_price

        save_data(products, 'products.json')
        print("Barang berhasil diperbarui.")
    else:
        print("Indeks barang tidak valid.")

# Fungsi untuk menghapus barang
def delete_product():
    show_products()
    product_index = int(input("Masukkan indeks barang yang ingin dihapus: "))

    if 0 <= product_index < len(products):
        products.pop(product_index)
        save_data(products, 'products.json')
        print("Barang berhasil dihapus.")
    else:
        print("Indeks barang tidak valid.")

# Fungsi untuk mencari barang
def search_product():
    keyword = input("Masukkan kata kunci: ")
    result = [i for i, product in enumerate(products) if keyword.lower() in product['name'].lower()]

    if result:
        for i in result:
            print(f"{i + 1}. {products[i]['name']} - Stok: {products[i]['stock']}, Harga: {products[i]['price']}")
    else:
        print("Barang tidak ditemukan.")

# Fungsi untuk memasukkan pesanan
def place_order(user):
    show_products()
    product_index = int(input("Masukkan indeks barang yang ingin dipesan \nKetik '0' untuk selesai: "))

    while product_index != 0:
        quantity = int(input("Masukkan jumlah barang: "))
        if 0 < product_index <= len(products) and quantity <= products[product_index - 1]['stock']:
            user['cart'].append({'product_index': product_index - 1, 'quantity': quantity})
            products[product_index - 1]['stock'] -= quantity
            save_data(products, 'products.json')
            print("Pesanan berhasil ditambahkan ke keranjang.")
        elif quantity > products[product_index - 1]['stock']:
            print("Stok barang tidak mencukupi.")
        else:
            print("Indeks barang tidak valid.")

        show_products()
        product_index = int(input("Masukkan indeks barang yang ingin dipesan \nKetik '0' untuk selesai: "))

# Fungsi untuk menghitung total pesanan
def calculate_total(user):
    total = sum([item['quantity'] * products[item['product_index']]['price'] for item in user['cart']])

    user['total'] = total
    print(f"Total Pesanan Anda: {total}")


# Fungsi untuk memberikan diskon
# Fungsi untuk memberikan diskon
def apply_discount(user):
    total = sum([item['quantity'] * products[item['product_index']]['price'] for item in user['cart']])
    print("SubTotal         : ", total)

    diskon = 0
    if total > 500000:
        diskon = 8/100
    elif total > 300000:
        diskon = 5/100
    elif total > 200000:
        diskon = 3/100
    elif total > 100000:
        diskon = 1/100

    user['discount'] = diskon

    total_akhir = total - (total * diskon)

    # Fungsi untuk menampilkan struk pembelian
    print("\n====== Struk Pembelian ======")

    # Iterate over items in the user's cart
    for item in user['cart']:
        product = products[item['product_index']]
        original_price = product['price']
        discounted_price = original_price - (original_price * diskon)

        subtotal = item['quantity'] * discounted_price
        print(f"{product['name']} - {item['quantity']} x {original_price} = {subtotal:.2f}")

    print("SubTotal         : ", total)
    print("Potongan Harga   : ", total * diskon)
    print("Total            : ", total_akhir)
    print("-------------------------------")

    bayar = int(input("Bayar            : "))  # Input Pembayaran
    kembalian = bayar - total_akhir
    print("Kembalian        : ", kembalian)
    print("-------------------------------")
    print("          Terima Kasih         ")
    print("-------------------------------")

# Fungsi untuk visualisasi hasil penjualan
def visualize_sales():
    if not products or not users:
        print("Data produk atau pengguna tidak tersedia.")
        return

    product_names = [product['name'] for product in products]
    product_sales = [sum([item['quantity'] for user in users.values() for item in user['cart'] if item['product_index'] == i]) for i in range(len(products))]

    plt.bar(product_names, product_sales, color='blue')  #tambahkan warna agar visualisasi lebih menarik
    plt.xlabel("Produk")
    plt.ylabel("Jumlah Terjual")
    plt.title("Grafik Penjualan Produk")
    plt.xticks(rotation=45, ha='right')  #Putar label sumbu x agar lebih mudah dibaca
    plt.tight_layout()  #Sesuaikan tata letak untuk tampilan yang lebih baik
    plt.show()

# Fungsi untuk laporan penjualan
def sales_report():
    if not users or not products:
        print("Data pengguna atau produk tidak tersedia.")
        return

    total_transactions = [item for user in users.values() for item in user['cart']]

    total_revenue = sum([
        sum([
            item['quantity'] * (products[item['product_index']]['price'] * (1 - user.get('discount', 0)))  # Apply discount
            for item in user['cart']
        ])
        for user in users.values()
    ])


    if total_transactions == 0:
        print("Tidak ada transaksi yang dilakukan.")
        return

    best_seller_product_index = max(range(len(products)), key=lambda x: sum([item['quantity'] for user in users.values() for item in user['cart'] if item['product_index'] == x]))

    best_seller_product = products[best_seller_product_index]['name'] if products else "Tidak ada produk terdaftar"

    print("\n===== Laporan Penjualan =====")
    # print(f"Total Transaksi: {total_transactions}")
    print(f"Total Transaksi: {len(total_transactions)}")
    print(f"Total Pendapatan: {total_revenue}")
    print(f"Produk Terlaris: {best_seller_product}")

# Inisialisasi data
users = load_data('users.json') or {}
products = load_data('products.json') or []

# Fungsi untuk menjalankan sistem
def main():
    while True:
        print("\n===== Aplikasi Kasir Sederhana =====")
        print("1. Login")
        print("2. Register")
        print("3. Keluar")

        choice = input("Pilih menu (1/2/3): ")

        if choice == '1':
            user = login()
            if user:
                if user['role'] == 'admin':
                    admin_menu(user)
                else:
                    user_menu(user)
        elif choice == '2':
            register()
        elif choice == '3':
            break
        else:
            print("Pilihan tidak valid. Coba lagi.")

# Menjalankan sistem
main()


===== Aplikasi Kasir Sederhana =====
1. Login
2. Register
3. Keluar
Pilih menu (1/2/3): 1
Masukkan username: user
Masukkan password: user

===== Menu User =====
1. Masukkan pesanan
2. Total Pesanan Anda
3. Diskon Potongan Harga
4. Keluar
Pilih menu (1/2/3/4): 1
1. blender - Stok: 497, Harga: 200000.0
Masukkan indeks barang yang ingin dipesan 
Ketik '0' untuk selesai: 1
Masukkan jumlah barang: 1
Pesanan berhasil ditambahkan ke keranjang.
1. blender - Stok: 496, Harga: 200000.0
Masukkan indeks barang yang ingin dipesan 
Ketik '0' untuk selesai: 0

===== Menu User =====
1. Masukkan pesanan
2. Total Pesanan Anda
3. Diskon Potongan Harga
4. Keluar
Pilih menu (1/2/3/4): 4

===== Aplikasi Kasir Sederhana =====
1. Login
2. Register
3. Keluar
Pilih menu (1/2/3): 1
Masukkan username: admin
Masukkan password: admin

===== Menu Admin =====
1. Tampilkan list barang
2. Tambah barang
3. Perbarui stok dan harga barang
4. Hapus barang
5. Cari barang
6. Visualisasi hasil penjualan
7. Laporan penjualan


KeyboardInterrupt: Interrupted by user